In [1]:
import os
import firebase_admin
import time
from tqdm.auto import tqdm
from firebase_admin import credentials, firestore

if '.gitignore' not in os.listdir():
    os.chdir('..')

In [2]:
cred = credentials.Certificate('website-cine-e77fb4ab2924.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

### Analysing movies from offline-database (in 2022)

In [25]:
data = db.collection("offline-database").stream() 
data_per_date = dict()
for elem in data:
    elem = elem.to_dict()
    data_per_date[elem['date']] = elem

movie_stream = db.collection("movie-info").stream()
movie_info = dict()
for elem in movie_stream:
    elem = elem.to_dict()
    movie_info[elem['id']] = elem

In [26]:
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

def plot_decades(decades_played, setxticks=False, cast_int=True):
    plt.clf()
    plt.figure(figsize=(12, 6), dpi=80)
    _ = plt.gca()
    if cast_int:
        plt.bar(decades_played[:,0].astype(int), decades_played[:,1].astype(int), width=2.5)
    else:
        plt.bar(decades_played[:,0], decades_played[:,1].astype(int), width=2.5)
    
    if setxticks:
        _, _ = plt.xticks(decades_played[:,0].astype(int))
    plt.show()

def get_counters_movies_and_dates(data_per_date, movie_info):
    movies_played = Counter()
    years_played = Counter()
    decades_played = Counter()
    directors_played = Counter()
    for date, data in data_per_date.items():
        movies_date = data["movies"]
        for mov in movies_date:
            year = mov.split("-")[-1]
            if year != "None":
                if (int(year)<=2018) and (int(year)>=1900):
                    movies_played.update([mov])
                    years_played.update([year])
                    decades_played.update([str(year)[:3]+"0"])
                    directors_played.update([movie_info[mov]["directors"]])
    return movies_played, years_played, decades_played, directors_played

movies_played, years_played, decades_played, directors_played = \
    get_counters_movies_and_dates(data_per_date=data_per_date, movie_info=movie_info)

print(directors_played.most_common(10))
print(movies_played.most_common(10))

# movies_played = np.array(movies_played.most_common(100))
# plot_decades(movies_played, setxticks=False, cast_int=False)

# years_played = np.array(sorted(years_played.items()))
# plot_decades(years_played, setxticks=False)

# decades_played = np.array(sorted(decades_played.items()))
# plot_decades(decades_played, setxticks=True)



[(None, 686), ('Wong Kar-Wai', 537), ('Pier Paolo Pasolini', 509), ('Stanley Kubrick', 414), ('François Truffaut', 376), ('Brian De Palma', 346), ('Kinuyo Tanaka', 343), ('Jean-Luc Godard', 330), ('Fritz Lang', 295), ('David Lynch', 288)]
[('mood-love-2000', 188), ('rex-studios-2008', 182), ('drive-my-car-2021', 163), ('dune-2021', 154), ('julie-12-chapitres-2021', 130), ('first-cow-2020', 128), ('illusions-perdues-2021', 125), ('mourir-peut-attendre-2021', 124), ('sommet-dieux-2021', 123), ('boite-noire-2020', 120)]


In [20]:
movies_played

Counter({'echiquier-vent-1976': 20,
         'il-varco-2018': 12,
         'mood-love-2000': 188,
         'l-avventura-1960': 18,
         'fedora-1978': 10,
         'tout-mere-1999': 18,
         'robocop-1987': 8,
         'peaux-de-vaches-1988': 17,
         'perfect-blue-1997': 4,
         'nuit-appartient-2007': 15,
         'after-hours-1985': 14,
         'chungking-express-1994': 48,
         'silence-agneaux-1991': 40,
         'buena-vista-social-club-1999': 3,
         'holy-motors-2012': 9,
         'nom-peuple-italien-1971': 21,
         'scoop-2006': 2,
         'ministere-peur-1944': 10,
         'decameron-ndeg-2-altre-novelle-del-boccaccio-1972': 1,
         'danton-1982': 2,
         'man-hunt-chasse-homme-1941': 10,
         'trois-chambres-manhattan-1965': 2,
         'belle-de-jour-1967': 15,
         'paprika-2005': 5,
         'vacances-1938': 6,
         'melodie-tueur-1978': 12,
         'paris-texas-1984': 47,
         'prisonniere-desert-1956': 12,
        